In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch
from torch.distributions.normal import Normal

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial State
        self.s_0 = nn.Parameter(torch.tensor([5.525, 5.1, 5.1,5.1], requires_grad=True)).requires_grad_(True)
        
        self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
        
        
        self.emission1=nn.Sequential(
                                    nn.Linear(16, 16),
                                     nn.LeakyReLU(0.2),
#                                      nn.Dropout(p=0.2),
                                    nn.Linear(16, 1))
        self.emission2=nn.Sequential(
                                    nn.Linear(16, 16),
                                    nn.LeakyReLU(0.2),
#                                      nn.Dropout(p=0.2),
                                    nn.Linear(16, 1))
        
        self.emission3=nn.Sequential(nn.Linear(2, 8),
                                   nn.LeakyReLU(0.2),
#                                      nn.Dropout(p=0.2),
                                    nn.Linear(8, 16),
                                    nn.LeakyReLU(0.2))
        self.emission4=nn.Sequential(nn.Linear(2, 8),
                                   nn.LeakyReLU(0.2),
#                                      nn.Dropout(p=0.2),
                                    nn.Linear(8, 16),
                                    nn.LeakyReLU(0.2))
        
        
        
        
        

    def forward(self, x,t,pred):

        x = F.softmax((x.clone()),dim=0)
        l=0.0

        x_temp=x.clone()
            
            
            
            
        o_1= self.emission((torch.tensor([1.0,0.0]).to(device)))
# #         o_1[1]=torch.exp(o_1[1])
        dist=Normal(o_1[0], o_1[1])
        
        o_1 = torch.exp(dist.log_prob(pred[0]))

        x_temp[0]=o_1*x[0].clone()

        o_1= self.emission((torch.tensor([2.0,0.0]).to(device)))
#         o_1[1]=torch.exp(o_1[1])
        dist=Normal(o_1[0], o_1[1])
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[1]=o_1.clone()*x[1].clone()

        o_1= self.emission((torch.tensor([3.0,0.0]).to(device)))
#         o_1[1]=torch.exp(o_1[1])
        dist=Normal(o_1[0], o_1[1])
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[2]=o_1*x[2].clone()
        
        o_1= self.emission((torch.tensor([4.0,0.0]).to(device)))
#         o_1[1]=torch.exp(o_1[1])
        dist=Normal(o_1[0], o_1[1])
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[3]=o_1*x[3].clone()
        
#         print(x_temp)
        


        x_temp=x_temp.clip(1e-6,1)
        
        x=x_temp.clone()/torch.sum(x_temp.clone())
        l+=torch.log(torch.sum(x_temp.clone()))# -0.01*state
        
        for i in range(1,t): 

            
            x=torch.matmul(x,F.softmax(self.T,dim=1))
            x_temp=x.clone()
            

            x_temp1=x.clone()
            o_1= self.emission((torch.tensor([1.0,i]).to(device)))
#             o_1[1]=torch.exp(o_1[1])
            dist=Normal(o_1[0], o_1[1])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[0]=o_1*x[0]

            o_1=self.emission((torch.tensor([2.0,i]).to(device)))
#             o_1[1]=torch.exp(o_1[1])
            dist=Normal(o_1[0], o_1[1])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[1]=o_1.clone()*x[1]
 
            o_1= self.emission((torch.tensor([3.0,i]).to(device)))
#             o_1[1]=torch.exp(o_1[1])
            dist=Normal(o_1[0], o_1[1])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[2]=o_1*x[2]

            o_1= self.emission((torch.tensor([4.0,i]).to(device)))
#             o_1[1]=torch.exp(o_1[1])
            dist=Normal(o_1[0], o_1[1])
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[3]=o_1*x[3].clone()
            
            
            
#             print(x_temp)
#             print(pred[i])
            x_temp=x_temp.clip(1e-6,1)

            x=x_temp.clone()/torch.sum(x_temp.clone())
#             print(x)
#             print("-----------------------------")
            l+=torch.log(torch.sum(x_temp.clone()))
        return l
    
    def transition(self,x):
        x=torch.matmul(x.to(device),F.softmax(self.T,dim=1))
        x_temp=x.clone()
        return x
    def Observation(self,x):

        o_1= self.emission((x))
        o_1[1]=torch.exp(o_1[1])

        return o_1
    def emission(self,x):
        x1= self.emission3((x))
        x= self.emission4((x))
        
        o_1= self.emission1((x))
        o_2= self.emission2((x1))
        o_2=torch.exp(o_2)

        return o_1,o_2
    
   

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'

cuda:0


In [5]:

T_matrix=torch.rand(3, 3).to(device)
T_matrix=torch.tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]], device=device)

# T_matrix[1,1]=8

# T_matrix[2,2]=8

# T_matrix[0,0]=8

T_matrix=T_matrix/torch.sum(T_matrix, 1).reshape(4,1)


s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_mean1=torch.tensor([0.1,0.02,0.03,0.04]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)







print(T_matrix)

print(s_0)

tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]])
tensor([1., 0., 0.])


In [6]:
def simulate_HMM(T_matrix,O_matrix,x,t):
    x=0
    
    
    t=0
    o=[]
#     o_1=torch.matmul(x.to(device),O_matrix.to(device))
# #         print(o_1)
    o.append(Normal(O_matrix_mean[x]+(O_matrix_mean1[x]*torch.tensor(t)),O_matrix_std[x]).sample().to(device))
    t=1
    for i in range(1,1000):
#         print(i)
        x=(torch.distributions.Categorical(T_matrix[x].to(device)).sample())
#         print(x)
        
        
        t+=1
        if x==0:
            o.append(Normal(O_matrix_mean[x]+(O_matrix_mean1[x]*torch.tensor(i)),O_matrix_std[x]).sample().to(device))
        if x==1:
            o.append(Normal(O_matrix_mean[x]+torch.exp(O_matrix_mean1[x]*torch.tensor(i)),O_matrix_std[x]).sample().to(device))
        if x==2:
#                 print(x)
            o.append(Normal(O_matrix_mean[x]+torch.exp(O_matrix_mean1[x]*torch.tensor(i))+torch.sin(torch.pi*torch.tensor(i)/5),O_matrix_std[x]).sample().to(device))
        if x==3:
            o.append(Normal(O_matrix_mean[x]+torch.exp(O_matrix_mean1[x]*torch.tensor(i))+torch.sin(torch.pi*torch.tensor(i)/10),O_matrix_std[x]).sample().to(device))
        t1=t
           
        if (t==50) :
            break
    
    
    
    o=torch.tensor(o).to(device)
    return o,t,t1



In [7]:
o=simulate_HMM(T_matrix,O_matrix_mean,torch.tensor([1.0,0,0.0]),1000)
o

(tensor([1.7412, 0.3638, 4.9690, 6.4506, 5.2028, 2.0542, 4.1758, 3.9717, 1.5313,
         0.8867, 4.7759, 6.0235, 3.8780, 6.5408, 3.8921, 3.9268, 3.9079, 1.4104,
         4.2620, 7.8937, 3.1240, 2.9964, 4.7481, 3.3664, 4.5002, 4.3918, 2.6616,
         3.9153, 3.3612, 4.0070, 4.3664, 5.1337, 4.9473, 5.8221, 6.1458, 5.6765,
         6.1171, 4.5809, 5.8496, 7.8438, 3.0200, 5.9959, 4.6692, 5.5863, 8.6105,
         7.6276, 7.7058, 7.7252, 6.9807, 8.0166]),
 50,
 50)

In [8]:
# def distribution_initalize(o):
#     s=torch.zeros(10)
#     for i in range(0,10):
#         s[i]=torch.sum(o==i)
        
#     return s#/torch.sum(s)
# def distribution_update(dist,o):

#     return dist + distribution_initalize(o)


In [9]:
# f=distribution_initalize(o[0])

def Expectation_diff(o_1):
    return torch.sum(o_1[0])

def var_diff(o_1):
    return torch.sum(o_1[1])

def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=torch.abs(e2-e1)
    return k


In [10]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import time

start_time = time.time()  

for batch_size in [5]:
    trial =0 
    while trial < 5:
        t=50
        # batch_size=20
        try: 
            pred1=[]
            for i in range(0,batch_size):
                pred1.append(simulate_HMM(T_matrix,O_matrix_mean,s_0,t))
            import torch.optim as optim
            net = Net()
            net.to(device)
            optimizer = optim.Adam(net.parameters(), lr=.1)
            scheduler =lr_scheduler.LinearLR(optimizer,start_factor=1, end_factor=0.001, total_iters=75)
            net.train()
    
            running_loss = 0.0
            o,t,t__1=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
            # dist=distribution_initalize(o)
            for epoch in range(100):  # loop over the dataset multiple times
    
    
    
    
                # zero the parameter gradients
                optimizer.zero_grad()
    
                # forward + backward + optimize
    
    
    
                loss=0.0
    
                for i in range(0,batch_size):
                    optimizer.zero_grad()
                    loss=0.0
    
            #         o,t=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
                    o,t,t_11=pred1[i]
                    pred=torch.zeros((1,t), dtype=torch.int32)
                    pred=o
            #         dist=distribution_update(dist,o)
                    loss-=net(net.s_0,t,pred.to(device))
    
                    l1_lambda = 0.001  # adjust the L1 regularization parameter as needed
                    l1_reg = torch.tensor(0.).to(device)
                    for param in net.parameters():
                        l1_reg += torch.norm(param, 1).to(device) 
                # #     print(dist)
                #     dist1=dist/torch.sum(dist)
                #     dist1=dist1.to(device)
                    f=0.0
                    for j in range(0,t):
                            o_1= net.emission((torch.tensor([1.0,j]).to(device)))
                            e1=Expectation_diff(o_1)
    
                            o_1=net.emission((torch.tensor([2.0,j]).to(device)))
                            e2=Expectation_diff(o_1)
    
                            o_1= net.emission((torch.tensor([3.0,j]).to(device)))
                            e3=Expectation_diff(o_1)
    
    
                            o_1= net.emission((torch.tensor([4.0,j]).to(device)))
                            e4=Expectation_diff(o_1)
                            f+=abs_diff([e1,e2,e3,e4])
                   
    
    
                    #+ l1_reg*l1_lambda
                #     print(loss)
                    loss=loss+ l1_reg*l1_lambda-0.001*f #+ 10*(torch.sum(t1*t2)/((torch.sum(t1**2)**0.5)*torch.sum(t2**2)**0.5) +torch.sum(t3*t2)/((torch.sum(t3**2)**0.5)*torch.sum(t2**2)**0.5)+torch.sum(t1*t3)/((torch.sum(t1**2)**0.5)*torch.sum(t3**2)**0.5) )
                    loss.backward()
                    optimizer.step()
                scheduler.step()
                    # print statistics
            # print(f'[{trial + 1}] loss: {loss/t}')
    
            # print('Finished Training')
    
            torch.save(pred1, "data_deep\ " + str(trial)+" _ "+str(batch_size)+ "_data.pt")
            torch.save(net.state_dict(),  "Model_deep\ " + str(trial)+" _ "+str(batch_size)+ "_model.pth")
            trial+=1
        except:
            print(f'[{trial + 1}] loss: {loss/t}')
            
            
            pass
end_time = time.time()  # Capture end time after the loop completes

print(f"Total execution time: {end_time - start_time} seconds")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10048\3316651859.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)


[1] loss: 0.0
[1] loss: 0.0
[1] loss: 0.0
[2] loss: 0.0
[4] loss: 0.0
[4] loss: 0.0
[5] loss: 0.0
Total execution time: 1130.5169491767883 seconds
